# IMU Analysis – BHI 2021

IMU Data Analysis for the Paper "Assessing the Influence of the Inner Clock on the Cortisol Awakening Response and Pre-Awakening Movement" (BHI 2021)

## Setup

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp
from biopsykit.utils.dataframe_handling import multi_xs
from biopsykit.stats import StatsPipeline

from carwatch_analysis.general_helper import describe_groups_df, subject_count_per_group
from carwatch_analysis.imu_helper import analysis_imu_features


%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
pg.options['round'] = 3

palette = bp.colors.fau_palette

theme_kwargs = {
    "context": "talk",
    "style": "ticks",
    "palette": palette
}
theme_kwargs_scale = {
    "context": "talk",
    "style": "ticks",
    "palette": palette,
    "font_scale": 1.25
}

sns.set_theme(**theme_kwargs)

#plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams['pdf.fonttype'] = 42
plt.close('all')

pad = 0.2

export = True
palette

In [ ]:
base_path = Path("../../..")
export_path = base_path.joinpath("exports")
result_path = base_path.joinpath("results/bhi_2021")
plot_path = result_path.joinpath("plots")
stats_path = result_path.joinpath("statistics")

paper_path = base_path.joinpath("../../Paper/CARInnerClock_BHI2021/img")

bp.utils.file_handling.mkdirs([result_path, plot_path, stats_path, paper_path])

## Data Import

### IMU Data

In [ ]:
imu_all = pd.read_csv(export_path.joinpath("imu_features_complete.csv"))
imu_all = imu_all.set_index(list(imu_all.columns[:-1]))
imu_all.head()

#### Feature Selection

In [ ]:
imu_all.index.get_level_values("imu_feature").unique()

In [ ]:
imu_data = imu_all.xs(('imu', 'last_30min'), level=['wakeup_type', 'time_span'])
filter_cols = ['max_position', 'max_60', 'number_60', 'mean_60', 'std_60']

list_dfs = []
imu_data = imu_data.unstack("imu_feature")
for col in filter_cols:
    list_dfs.append(imu_data.filter(like=col))
imu_data = pd.concat(list_dfs, axis=1).stack()
imu_data

#### Outlier Removal

In [ ]:
outlier_mask = imu_data.groupby('imu_feature').apply(lambda df: ((df - df.mean())/df.std()).abs() > 3)['data']

print("Number of nights before outlier removal: {}".format(len(imu_data.unstack('imu_feature'))))

imu_data = imu_data.loc[~outlier_mask]
imu_data = imu_data.unstack('imu_feature').dropna().stack()

print("Number of nights after outlier removal: {}".format(len(imu_data.unstack())))

In [ ]:
max_pos = imu_data.filter(like='ss_max_position', axis=0)
imu_data.loc[max_pos.index] = max_pos.transform(lambda df: -(1 - df) * 30)

## Plots and Statistics

In [ ]:
condition_order = ['Spontaneous', 'Known Alarm', 'Unknown Alarm']

### Class Distribution

In [ ]:
subject_count_per_group(imu_data, "condition", order=condition_order)

In [ ]:
describe_groups_df(imu_data, "condition", order=condition_order)

### IMU Data

#### Statistics

In [ ]:
pipeline = analysis_imu_features(imu_data, variable="condition", test_type='welch_anova')
pipeline.display_results(posthoc=True, sig_only='posthoc')
if export:
    pipeline.export_statistics(stats_path.joinpath("analysis_imu_features_condition.xlsx"))

#### Result Table for Paper

In [ ]:
index_labels = {
    'ss_number_60': r"$\lvert sp \rvert$",
    'ss_max_60': r"$sp_{max}$",
    'ss_max_position': r"$t(sp_{max})$",
    'ss_mean_60': r"$\mu_{sp}$",
    'ss_std_60': r"$\sigma_{sp}$",
}
pipeline.df_to_latex('welch_anova', index_labels)

#### Plots

In [ ]:
hue = 'condition'

ylabels = {
    'ss_max_60': "Duration [s]",
    'ss_max_position': "Time before Awak. [min]",
    'ss_std_60': "Duration [s]"
}

xticklabels = {
    'ss_max_60': r"$sp_{max}$",
    'ss_max_position': r"$t(sp_{max})$",
    'ss_std_60': r"$\sigma_{sp}$",
}

features = ['ss_max_60', 'ss_max_position', 'ss_std_60']
box_pairs, pvals = pipeline.sig_brackets('posthoc', stats_type='between', plot_type='multi', x="imu_feature", subplots=True, features=features)

sns.set_theme(**theme_kwargs_scale)
fig, axs = plt.subplots(figsize=(12,5), ncols=len(features))
bp.plotting.multi_feature_boxplot(
    x='imu_feature', 
    y='data',
    group="imu_feature",
    data=imu_data, 
    hue=hue, 
    features=features, 
    hue_order=condition_order, 
    ylabels=ylabels, 
    xticklabels=xticklabels, 
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvals,
        'text_offset': -8
    }, 
    axs=axs, 
    legend_fontsize='small', 
    legend_orientation="horizontal", 
    legend_loc="upper center",
    width=0.9,
    saturation=1.0
)

axs[1].set_ylim([-32.5, 2.5])
fig.tight_layout(rect=(0, 0, 1.0, 0.89), pad=pad)

if export:
    [fig.savefig(p.joinpath("img_boxplots_imu_condition.pdf"), transparent=True) for p in [plot_path, paper_path]]